In [ ]:
import pandas as pd
import plotly.express  as px
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix,  classification_report




# Parte 2

# Análise inicial, Tratamento e Pré-processamento dos dados (Deliverable 3)

Primeiramente, realizamos uma análise primária no dataset escolhido para a parte 2 utilizando a biblioteca `pandas` para reconhecer nossos `atributos` e `classes`. Através da função `describe` evidenciamos as estatísticas e vemos alguns valores inconsistentes que posteriormente são tratados para não afetar negativamente nosso modelo com dados incongruentes. Exemplo: idade negativa

In [ ]:
base_credit = pd.read_csv("datasets/credit_data.csv")
base_credit.describe()

Nos trecho abaixo, apagamos somente os dados incorretos e preenchemos as lacunas com a média do resto dos dados. Fizemos o mesmo para linhas com dados faltantes.

In [ ]:
base_credit3 = base_credit.drop(base_credit[base_credit['age']< 0].index) # outro método é preencher-los manualmente

In [ ]:
print('Média com os valores inconsistentes: ',base_credit['age'].mean()) 
print('Média com valores inconsistentes removidos: ',base_credit3['age'].mean()) 

In [ ]:
base_credit.loc[base_credit['age'] < 0, 'age'] = 40.92 # Preenchimento dos valores inconsistentes com a média

In [ ]:
base_credit.isnull().sum()

In [ ]:
base_credit.loc[pd.isnull(base_credit['age'])]

In [ ]:
base_credit['age'].fillna(base_credit['age'].mean(), inplace = True)

## Divisão entre Previsores e Classes

In [ ]:
X_credit = base_credit.iloc[:, 1:4].values
X_credit

In [ ]:
y_credit = base_credit.iloc[:, 4].values
y_credit

## Padronizaçao

Abaixo são realizadas algumas etapas de pré-processamento, essas etapas são comumente usadas para normalizar os dados, garantindo que eles estejam na mesma escala antes de serem usados em algoritmos de aprendizado de máquina. A padronização é uma técnica que transforma os dados de forma que eles tenham média zero e desvio padrão igual a um. Isso pode ser útil para algoritmos que assumem que os dados estão normalmente distribuídos e têm a mesma escala.

In [ ]:
scaler_credit = StandardScaler()
X_credit = scaler_credit.fit_transform(X_credit)
X_credit

## Divisão entre treinamento e teste (Deliverable 3)

Neste trecho de código, está sendo realizado o particionamento dos dados em conjuntos de treinamento e teste usando a função `train_test_split`. Sendo 70% do dataset para treino e 30% para teste.

Esse particionamento dos dados é uma etapa importante para garantir que o modelo seja avaliado corretamente e que possa generalizar bem para novos dados.

In [ ]:
X_credit_train, X_credit_teste, y_credit_train, y_credit_teste = train_test_split(X_credit, y_credit, test_size = 0.30, random_state=0)
X_credit_train.shape

# Primeiro modelo: Naive Beans

Antes de treinar o modelo vamos verificar quais os melhores hipeparâmetros para o algoritimo SVM através da classe `GridSearchCV` da biblioteca `scikitlearn`. O GridSearchCV é uma técnica popular em aprendizado de máquina usada para encontrar os melhores hiperparâmetros para um modelo, funciona através da criação de um "grid" de hiperparâmetros possíveis e testa todas as combinações possíveis desses hiperparâmetros, usando validação cruzada para avaliar o desempenho de cada combinação. 

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV

# Cria uma instância do classificador Naive Bayes com a distribuição Gaussiana (GaussianNB)
naive_credit_data = GaussianNB() 

# Define os hiperparâmetros para testar
param_grid = {
    'var_smoothing': np.logspace(0,-9, num=200)
}

# Cria o objeto GridSearchCV
grid_search = GridSearchCV(estimator=naive_credit_data, param_grid=param_grid, cv=5)

# Treina o modelo com o Grid Search
grid_search.fit(X_credit_train, y_credit_train)

# Obtem os melhores hiperparâmetros encontrados
best_params = grid_search.best_params_

# Imprime os melhores hiperparâmetros
print("Melhores hiperparâmetros encontrados:")
print(best_params)

In [ ]:

naive_credit_data = GaussianNB(var_smoothing=8.504489341802669e-05) # Cria uma instância do classificador Naive Bayes com a distribuição Gaussiana (GaussianNB)
naive_credit_data.fit(X_credit_train, y_credit_train) # Treina o modelo Naive Bayes com os dados de treinamento padronizados e seus rótulos

In [ ]:
naive_credit_data = GaussianNB(var_smoothing=8.504489341802669e-05) # Cria uma instância do modelo NB com o valor encontrado como melhor para hiperparâmetro var_smoothing.
naive_credit_data.fit(X_credit_train, y_credit_train) # Treina o modelo 
NB_predict = naive_credit_data.predict(X_credit_teste) # Faz previsões usando o modelo Naive Bayes treinado nos dados de teste e armazena na variavel predict


In [ ]:
y_true = y_credit_teste  # Rótulos verdadeiros
y_pred = NB_predict  # Previsões do modelo

# Calcula a matriz de confusão
cm = confusion_matrix(y_true, y_pred)

# Plota a matriz de confusão como um mapa de calor (heatmap)
plt.figure(figsize=(8, 6))
sn.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Previsão')
plt.ylabel('Rotulo Verdadeiro')
plt.title('Matriz de Confusão')
plt.show()

In [138]:

print(classification_report(y_credit_teste, NB_predict))
print(accuracy_score(y_credit_teste, NB_predict))

              precision    recall  f1-score   support

           0       0.94      0.98      0.96       524
           1       0.84      0.61      0.70        76

    accuracy                           0.94       600
   macro avg       0.89      0.79      0.83       600
weighted avg       0.93      0.94      0.93       600

0.935


# Segundo modelo: Decision Tree

In [137]:
from sklearn.tree import DecisionTreeClassifier

# Cria uma instância do classificador Decision Tree
tree_credit = DecisionTreeClassifier()

# Define os hiperparâmetros para testar
param_grid = {
    'criterion': ['entropy', 'gini'],
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5]
}

# Cria o objeto GridSearchCV
grid_search = GridSearchCV(estimator=tree_credit, param_grid=param_grid, cv=5)

# Treina o modelo com o Grid Search
grid_search.fit(X_credit_train, y_credit_train)

# Obtem os melhores hiperparâmetros encontrados
best_params = grid_search.best_params_

# Imprime os melhores hiperparâmetros
print("Melhores hiperparâmetros encontrados:")
print(best_params)

Melhores hiperparâmetros encontrados:
{'criterion': 'entropy', 'max_depth': 15, 'min_samples_leaf': 1, 'min_samples_split': 2}


In [ ]:
tree_credit = DecisionTreeClassifier(criterion='entropy', random_state=0, min_samples_split=2, min_samples_leaf=1, max_depth=10)
tree_credit.fit(X_credit_train, y_credit_train)
tree_predict = tree_credit.predict(X_credit_teste)


In [ ]:
y_true = y_credit_teste # Rótulos verdadeiros
y_pred = tree_predict  # Previsões do modelo

# Calcula a matriz de confusão
cm = confusion_matrix(y_true, y_pred)

# Plota a matriz de confusão como um mapa de calor (heatmap)
plt.figure(figsize=(8, 6))
sn.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Previsão')
plt.ylabel('Rotulo Verdadeiro')
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
from sklearn import tree
import matplotlib.pyplot as plt
previsores = ['income', 'age', 'loan']
fig, axes = plt.subplots(nrows = 1,ncols = 1,figsize = (10,10))
tree.plot_tree(tree_credit, feature_names=previsores, class_names=['0','1'], filled=True);
fig.savefig('tree_credit.png')

In [139]:
print(classification_report(y_credit_teste, tree_predict))
print(accuracy_score(y_credit_teste, tree_predict))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       524
           1       0.92      0.95      0.94        76

    accuracy                           0.98       600
   macro avg       0.96      0.97      0.96       600
weighted avg       0.98      0.98      0.98       600

0.9833333333333333


# Terceiro modelo: SVM (Support Vector Machine)

In [ ]:
from sklearn import svm

# Cria uma instância do classificador SVM
svm_credit = svm.SVC()

# Define os hiperparâmetros para testar
param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

# Cria o objeto GridSearchCV
grid_search = GridSearchCV(estimator=svm_credit, param_grid=param_grid, cv=5)

# Treina o modelo com o Grid Search
grid_search.fit(X_credit_train, y_credit_train)

# Obtem os melhores hiperparâmetros encontrados
best_params = grid_search.best_params_

# Imprime os melhores hiperparâmetros
print("Melhores hiperparâmetros encontrados:")
print(best_params)

In [ ]:

svm_credit = svm.SVC(C=100, gamma='auto', kernel='rbf')
svm_credit.fit(X_credit_train, y_credit_train)
svm_predict = svm_credit.predict(X_credit_teste)

In [ ]:
y_true = y_credit_teste # Rótulos verdadeiros
y_pred = svm_predict  # Previsões do modelo

# Calcula a matriz de confusão
cm = confusion_matrix(y_true, y_pred)

# Plota a matriz de confusão como um mapa de calor (heatmap)
plt.figure(figsize=(8, 6))
sn.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Previsão')
plt.ylabel('Rotulo Verdadeiro')
plt.title('Matriz de Confusão')
plt.show()

In [140]:
print(classification_report(y_credit_teste, svm_predict))
print(accuracy_score(y_credit_teste, svm_predict))


              precision    recall  f1-score   support

           0       0.99      1.00      0.99       524
           1       0.99      0.91      0.95        76

    accuracy                           0.99       600
   macro avg       0.99      0.95      0.97       600
weighted avg       0.99      0.99      0.99       600

0.9866666666666667


# Conclusão
Obtivemos resultados positivos acima dos 90% de acurácia nos 3 modelos testados, ficando mais ou menos empatado entre os modelos Decision Tree e SVM, então abaixo iremos analisar as estatísticas obtidas nos dois modelos e decidir qual o melhor:

### Acurácia (Accuracy):

- Decision Tree: 98%
- SVM: 99%

O modelo SVM obteve uma acurácia ligeiramente maior em comparação com a árvore de decisão. Isso significa que o SVM classifica corretamente uma maior proporção de exemplos em comparação com a árvore de decisão.

### Precisão (Precision):

- Para a classe 0 (negativa):
Decision Tree: 99%
SVM: 99%
- Para a classe 1 (positiva):
Decision Tree: 92%
SVM: 99%

O SVM obteve uma precisão ligeiramente maior para a classe 1 (positiva), o que significa que o SVM comete menos falsos positivos em comparação com a árvore de decisão.

### Revocação (Recall):

- Para a classe 0 (negativa):
Decision Tree: 99%
SVM: 100%
- Para a classe 1 (positiva):
Decision Tree: 95%
SVM: 91%

A árvore de decisão obteve uma revocação ligeiramente maior para a classe 1 (positiva), o que significa que a árvore de decisão identifica corretamente uma maior proporção de exemplos positivos em comparação com o SVM.

### F1-Score:

- Para a classe 0 (negativa):
Decision Tree: 99%
SVM: 99%
- Para a classe 1 (positiva):
Decision Tree: 94%
SVM: 95%

O SVM obteve um F1-score ligeiramente maior para a classe 1 (positiva), o que representa um equilíbrio entre precisão e revocação.


Em resumo, o SVM parece ter um desempenho ligeiramente melhor em termos de acurácia geral, enquanto a árvore de decisão obteve um desempenho melhor em termos de revocação para a classe positiva. A escolha entre esses modelos dependerá das necessidades específicas do problema e das métricas de avaliação prioritárias.

# Sample Complexity (Deliverable 4)